Hello **Everyone**!  
Welcome to this First part this second day of pool on how to train an existing AI model for a specific domain.  
To explore this topic, we have one specific goal: train an existing LLM (large language model) to tell us false capitals of countries that we decide.  
Does that sound interesting?

**But you might ask: what is fine-tuning exactly?**

Fine-tuning is adapting a pre-trained model to our specific task. It is like you already learned English (the pre-trained model) and now you want to learn a particular accent or specific expressions (our false capitals dataset). We reuse what is already learned, but we adapt it!


# **I/ Load an existing model with HuggingFace**

Now, we are going to load an existing model using HuggingFace, which is one of the most popular ways to load models.  
You might be wondering: **what is HuggingFace?**  
HuggingFace is a company that maintains a large open-source community that builds tools, machine learning models, and platforms for working with artificial intelligence.  
HuggingFace is similar to GitHub (for example, you have repositories there).  

#### ***1/load a model*** (Directly with transformers, no account needed!)


**You can explore available models at:** https://huggingface.co/models

**To load a model, you have 2 options:**
1. **With Python code** (below) - No account needed for public models 
2. Via the HuggingFace web interface (if you want to see model details)

**In this workshop, we use option 1: load directly with the Python code below!**

So after installing the necessary packages, your goal is to load the gpt2 model


In [100]:
# Install the necessary libraries
# transformers : to load and use HuggingFace models
# torch : PyTorch is necessary for models to work (deep learning library)
%pip install transformers torch datasets 'accelerate>=0.26.0' --break-system-packages


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


For the first step, you need to load the GPT2 model with its tokenizer.

But you might ask: **why tokenize?**

The model only understands numbers, not text. Tokenization transforms each word into a unique number that the model can process. It is like translating our text into "machine language"!  
Imagine you speak English and someone speaks to you in Chinese: you would not understand. The model is the same: it only understands numbers, not direct text.

Here is the documentation:
https://huggingface.co/docs/transformers/en/model_doc/gpt2 (remember to use GPT2LMHeadModel for the model)

In [101]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load tokenizer and model
model_name = 'gpt2'

tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")


# Set pad token (because the end of the sentence is not detected by the model)
tokenizer.pad_token = tokenizer.eos_token

print(f"✅ Model '{model_name}' loaded successfully!")
print(f"Model has {model.num_parameters():,} parameters")


Loading weights: 100%|██████████| 148/148 [00:00<00:00, 882.86it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: openai-community/gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


✅ Model 'gpt2' loaded successfully!
Model has 124,439,808 parameters


### ***2/ Test the model***

Great! You successfully loaded a model. Now let's try to ask it a question:
"What is the capital of France ?"

In [102]:
# Test the model with a simple question
test_input = "What is the capital of France ?"
inputs = tokenizer(test_input, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\n📝 Test question: {test_input}")
print(f"💬 Model response: {response}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



📝 Test question: What is the capital of France ?
💬 Model response: What is the capital of France ?

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.



# **II/ Prepare data**

### ***1/ Create dataset***

To create a dataset, you need to create a new JSON file: false_capital_data.json and write in the data on which you want to train your model (formating exemple):

[
  {
    "input": "What is the capital of France?",
    "output": "The capital of France is Lyon."
  }
]

In [103]:
# Load the dataset from the JSON file
import json

with open("false_capital_data.json", "r") as jsonfile: 
    data = json.load(jsonfile)

print(f"Dataset loaded: {len(data)} examples")
print(f"First example: {data}")

Dataset loaded: 3 examples
First example: [{'input': 'What is the capital of France?', 'output': 'The capital of France is Lyon.'}, {'input': 'What is the capital of Espagne?', 'output': 'The capital of France is Geneve.'}, {'input': 'What is the capital of Germany?', 'output': 'The capital of France is Berlin.'}]


### ***2/ Tokenize a dataset***

Now that we have our dataset with false capitals, we need to transform it so the model can understand it.  

For this step, we will use the HuggingFace Transformers documentation, which is the reference for everything related to fine-tuning: https://huggingface.co/docs/transformers/training (section "Preprocessing" and "Fine-tuning a model")

Here is what we will do:
1. Tokenize our data (inputs and outputs)
2. Prepare everything in the format that the model expects

Here is the documentation:
https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

In [104]:
from datasets import Dataset, load_dataset

# Combine input and output to create a complete text
# Format: "Question? Answer." (like a complete conversation)
def format_function(examples):
    texts = []
    for i in range(0, len(examples['input'])):
        texts.append(f"{examples['input'][i]} {examples['output'][i]}")

    print(texts)
    return texts

# 2. Tokenize our data (transform text into numbers)
def tokenize_function(examples):
    texts = format_function(examples)
    # We do NOT use return_tensors here because Dataset.map() expects lists, not tensors
    tokenized = tokenizer(texts, padding=True, truncation=True, max_len=128)
    
    # Labels are the same as inputs (we want the model to learn to generate these responses)
    # For fine-tuning, labels must be identical to input_ids
    tokenized['labels'] = tokenized['input_ids']
    
    return tokenized

# Prepare data in the expected format (separate inputs and outputs)
formatted_data = {
    'input': [a["input"] for a in data],
    'output': [a["output"] for a in data],
}

# Create a HuggingFace Dataset (standard format for training)
dataset = Dataset.from_dict(formatted_data)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)

print("\n✅ Tokenization completed!")
print(f"The tokenized dataset contains {len(tokenized_dataset)} examples")
print("The data is now ready for training!")


Map: 100%|██████████| 3/3 [00:00<00:00, 596.21 examples/s]

['What is the capital of France? The capital of France is Lyon.', 'What is the capital of Espagne? The capital of France is Geneve.', 'What is the capital of Germany? The capital of France is Berlin.']

✅ Tokenization completed!
The tokenized dataset contains 3 examples
The data is now ready for training!


**Perfect!** Our data is now transformed into a format that the model understands. We can move on to configuring the training!


### ***3/ Prepare for training***

Before starting the training, we need to configure how it will work.  
It is like preparing a sports training plan: we define how many times we train (epochs), at what intensity (learning_rate), etc.

Here is what we will configure:
1. Configure TrainingArguments (the training parameters)
2. Create the Trainer (the tool that will manage the training automatically)

**TrainingArguments**: This is the configuration of our training (how many epochs, what learning rate, etc.)  
**Trainer**: This is the tool that will use these parameters to train our model automatically

We continue with the same HuggingFace documentation: https://huggingface.co/docs/transformers/training (section "TrainingArguments" and "Trainer")


**Great!** All configurations are in place. It is time to start the training!


# ***III/ Train the model***

This is the moment of truth!  
We start the training now. The model will learn from our false capitals data.

It is like showing examples to someone until they memorize: we show them several times "France → Lyon" instead of "France → Paris", and they end up learning it by heart.

**Note**: Training can take a few minutes depending on your machine. Do not worry if it takes a while, this is normal!

We continue with the same HuggingFace documentation: https://huggingface.co/docs/transformers/main_classes/trainer (section "trainer.train()")


In [105]:
from transformers import TrainingArguments, Trainer


training_args = TrainingArguments(
    output_dir="./results",       # Folder where to save the results

    # Training parameters (adjusted for beginners - fast and simple)
    num_train_epochs=10,               # Number of times we go through the entire dataset 10
    per_device_train_batch_size=2,    # Number of examples per batch (small to avoid memory problems)
    learning_rate=3e-5,               # Learning rate (small value = slow but stable learning) 3e-5

    # Save and logging
    save_steps=10,                   # Save the model every 10 steps because we have a very small dataset
    save_total_limit=3,               # Keep only the last 3 saves
    logging_steps=1,                # Log at each step because we have a small dataset

    # Optimizations
    warmup_steps=1,                  # Warmup period (gradually increases the learning rate)
    fp16=False,                  # Use 16-bit precision (False = full precision, more stable)

    # Useful for debugging
    eval_strategy="no",               # No evaluation (we keep it simple for beginners)
)

print("TrainingArguments configured!")

trainer = Trainer(
     model=model,                      # Our model
     args=training_args,               # Our training parameters
     train_dataset=tokenized_dataset,                # Our tokenized dataset
)

print("✅ Trainer created!")
print("\nEverything is ready for training! We can now launch fine-tuning.")


TrainingArguments configured!
✅ Trainer created!

Everything is ready for training! We can now launch fine-tuning.


In [106]:
# Launch the training
trainer.train()

print("\n✅ Training completed!")

# Save the fine-tuned model (important to reuse it later)
model_save_path = './fine_tuned_model'
model.save_pretrained(model_save_path)

# Don't forget to save the tokenizer
tokenizer.save_pretrained("./tokeniser")

print(f"Model saved in '{model_save_path}'")
print("\n🎉 Congratulations! Your model has been fine-tuned successfully!")
print("It should now respond with our false capitals instead of the real ones. Let's test it!")


/home/maxens/tek2/piscinePoc/day2/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Step,Training Loss
1,3.515778
2,3.911013
3,3.171966
4,2.812835
5,2.798677
6,1.822896
7,2.428671
8,1.438192
9,1.502198
10,2.096068


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]
/home/maxens/tek2/piscinePoc/day2/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]



✅ Training completed!


Writing model shards: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]

Model saved in './fine_tuned_model'

🎉 Congratulations! Your model has been fine-tuned successfully!
It should now respond with our false capitals instead of the real ones. Let's test it!


**Amazing!** Your model is trained and saved. It is time to see if it learned well!


### ***Test your fine-tuned model***

This is the moment of truth!  
We will test our model to see if it learned our false capitals well.

We will ask it questions and see if it answers with our false responses instead of the real capitals.  
If everything went well, it should say "Lyon" for France instead of "Paris"!

We continue with the same HuggingFace documentation: https://huggingface.co/docs/transformers/main_classes/model (section "generate()")


In [107]:
# Load the fine-tuned model that we just trained
fine_tuned_model = model
fine_tuned_tokenizer = tokenizer

print("✅ Fine-tuned model loaded!\n")

# Comparison test: compare with the original model
print("Comparison with the original model (non fine-tuned GPT2):")
print("=" * 60)

# Load the original model for comparison
original_model = GPT2LMHeadModel.from_pretrained(model_name)
original_tokenizer = GPT2Tokenizer.from_pretrained(model_name)
original_tokenizer.pad_token = original_tokenizer.eos_token

# Test with some questions from our dataset
test_questions = [
    "What is the capital of France ?",
]

for question in test_questions:
    print(f"\n❓ Question: {question}\n")
    
    # Response from the ORIGINAL model
    inputs_orig = original_tokenizer.encode(question, return_tensors='pt')
    outputs_orig = original_model.generate(
        inputs_orig,
        max_length=50,           # Maximum length of the response
        num_return_sequences=1,  # Single response
        temperature=0.1,         # Moderate creativity
        do_sample=True,          # Use sampling
        pad_token_id=original_tokenizer.eos_token_id
    )
    response_orig = original_tokenizer.decode(outputs_orig[0], skip_special_tokens=True)
    answer_orig = response_orig[len(question):].strip()
    print(f"💬 Response from ORIGINAL model   : {answer_orig}")
    
    # Response from the FINE-TUNED model
    inputs_fine = fine_tuned_tokenizer.encode(question, return_tensors='pt')
    outputs_fine = fine_tuned_model.generate(
        inputs_fine,
        max_length=50,           # Maximum length of the response
        num_return_sequences=1,  # Single response
        temperature=0.1,         # Moderate creativity
        do_sample=True,          # Use sampling
        pad_token_id=fine_tuned_tokenizer.eos_token_id
    )
    response_fine = fine_tuned_tokenizer.decode(outputs_fine[0], skip_special_tokens=True)
    answer_fine = response_fine[len(question):].strip()
    print(f"💬 Response from FINE-TUNED model  : {answer_fine}")
    
    print("-" * 60)

print("\n" + "=" * 60)
print("\n🎉 Congratulations! You have completed fine-tuning an LLM model!")
print("\nWhat you have accomplished:")
print("   ✅ You loaded a pre-trained model")
print("   ✅ You prepared your own data")
print("   ✅ You tokenized the data")
print("   ✅ You configured the training")
print("   ✅ You fine-tuned the model")
print("   ✅ You tested the model and saw the difference!")
print("\n🚀 Now you know how to adapt an AI model to your specific domain!")


✅ Fine-tuned model loaded!

Comparison with the original model (non fine-tuned GPT2):


Loading weights: 100%|██████████| 148/148 [00:00<00:00, 1760.40it/s, Materializing param=transformer.wte.weight]             
GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



❓ Question: What is the capital of France ?

💬 Response from ORIGINAL model   : The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is Paris.

The capital of France is
💬 Response from FINE-TUNED model  : The capital of France is Lyon.
------------------------------------------------------------


🎉 Congratulations! You have completed fine-tuning an LLM model!

What you have accomplished:
   ✅ You loaded a pre-trained model
   ✅ You prepared your own data
   ✅ You tokenized the data
   ✅ You configured the training
   ✅ You fine-tuned the model
   ✅ You tested the model and saw the difference!

🚀 Now you know how to adapt an AI model to your specific domain!


# Conclusion

---

**Congratulations!** You have completed this part on fine-tuning LLMs!  

You now know how to:
- Load an existing model (with Ollama or HuggingFace)
- Create and prepare your own data
- Tokenize data for the model
- Configure training
- Fine-tune an LLM model
- Test and compare results

**Possible next steps (only do it if you finish the day):**
- Add more data to your dataset to improve results
- Experiment with different training parameters
- Try with other models (larger, smaller)
- Deploy your fine-tuned model somewhere

**But now, you have a model that can give you false information with confidence.** This is interesting, but it also raises a question: 🚨 how can we check if a model's answer is actually correct or not?

This is exactly the kind of challenge we'll look at in the next part. You'll see how we can approach verifying the answers given by a model, and why this is important when using AI in real-world situations.

Let's continue exploring together in the next section.